In [1]:
# Instalo bibliotecas
!pip install bs4 --quiet

In [2]:
# Importo
import pandas as pd
import pickle 
import re 
import requests
import time
import concurrent.futures
import sys

from bs4 import BeautifulSoup

In [3]:
# Inicializacion de constantes

# Defino cantidad de topicos y paginas
topics = ['economia', 'el-mundo', 'sociedad']
pages = range(1, 5)

# Defino headers del request
request_headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.84 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
}

In [4]:
# Creo funcion que retorna las noticias de un topico determinado
RegExp = re.compile('\\n*')

def retrieve_topic_news(topic):
  topic_news = []
  for page in pages:
    request = requests.get(f'https://www.pagina12.com.ar/secciones/{topic}?page={page}', headers=request_headers)
    soup = BeautifulSoup(request.text, 'html.parser')
    articles = soup.find_all('article', class_='article-item article-item--teaser ')
    for article in articles:
      anchor = article.find('a', class_='p12-separator--left--primary')
      if anchor is not None:
        article_detail = requests.get(f'https://www.pagina12.com.ar/{anchor["href"]}', headers=request_headers)
        soup_detail = BeautifulSoup(article_detail.text, 'html.parser')
        paragraphs = soup_detail.find('div', class_='article-main-content article-text ').find_all('p')
        dates = soup_detail.find('div',class_='article-info')
        if dates is not None:
          dates = dates.find('span').string
          news = ''
          for paragraph in paragraphs:
            if paragraph.string is not None:
              news += paragraph.string
          if news != '' and not RegExp.fullmatch(news):
            topic_news.append({'url':f'https://www.pagina12.com.ar/{anchor["href"]}','paragraph':news,'date':dates})
    time.sleep(3)
  return {topic: topic_news}

In [5]:
with concurrent.futures.ThreadPoolExecutor() as executor:
  topics_results = executor.map(retrieve_topic_news, topics)

  topics_news = dict()
  for topic_result in topics_results:
    topics_news.update(topic_result)

In [6]:
topics_news

{'economia': [{'date': '10 de septiembre de 2021',
   'paragraph': 'En comparación con la anterior jornada hábil, donde el blue cotizaba a $183,00 para la compra y $187,00 para la venta.Se denomina Dólar Blue al dólar que se consigue en el mercado negro o mercado paralelo. En Argentina se utilizan eufemismos para referirse a este dólar y se lo llama además de blue, dólar paralelo o dólar negro. Este dólar no se compra en bancos o casas de cambio oficiales, sino que se comercializa en “cuevas” o los venden los tradicionales “arbolitos” en calles principales del microcentro. La cotización del dólar blue es generalmente superior a la cotización del dólar oficial y la brecha suele aumentar a medida que aumenta el control de cambios o se incrementan las restricciones para la compra de dólares al precio oficial. El mercado del dólar blue es insignificante en monto respecto al resto de las operaciones con divisas. Tiene, fundamentalmente, influencia en la generación de expectativas sociales y

In [8]:
sys.setrecursionlimit(300000)

with open('TP1.pickle', 'wb') as handle:
    pickle.dump(topics_news, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [11]:
from google.colab import files
files.download('./TP1.pickle')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>